Dataset: http://data.torontopolice.on.ca/datasets/bicycle-thefts

Variables/Attributes observation and relation between attributes and status (recovered or not)

Variables in data: Division, City, Location_Type, Premise_Type, Neighbourhood, Hood_ID, Lat, Long

In [2]:
#Import all the packages we will need 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import gmplot
# For improved table display in the notebook
import IPython
from IPython.display import Image

In [2]:
#Download dataset 
#https://opendata.arcgis.com/datasets/22bfc3619d69447fadd984fcf77a5550_0.csv

url = "https://opendata.arcgis.com/datasets/22bfc3619d69447fadd984fcf77a5550_0.csv"
df = pd.read_csv(url)

df.head()

,X,Y,Index_,event_unique_id,Primary_Offence,Occurrence_Date,Occurrence_Year,Occurrence_Month,Occurrence_Day,Occurrence_Time,...,Bike_Type,Bike_Speed,Bike_Colour,Cost_of_Bike,Status,Neighbourhood,Hood_ID,Lat,Long,ObjectId
0,-79.367424,43.652412,1801,GO-20149004060,THEFT UNDER,2014-06-13T04:00:00.000Z,2014,6,13,11:30,...,OT,24,BLK,800.0,STOLEN,Moss Park,73,43.652412,-79.367424,1001
1,-79.357460,43.659527,1802,GO-20149004061,THEFT UNDER,2014-06-13T04:00:00.000Z,2014,6,13,11:45,...,EL,32,ONG,1200.0,STOLEN,Regent Park,72,43.659527,-79.357460,1002
2,-79.383110,43.661373,1803,GO-20149004063,THEFT UNDER,2014-05-02T04:00:00.000Z,2014,5,2,12:45,...,OT,21,BLK,900.0,STOLEN,Church-Yonge Corridor,75,43.661373,-79.383110,1003
3,-79.290672,43.678585,1804,GO-20149004064,THEFT UNDER,2014-06-13T04:00:00.000Z,2014,6,13,16:30,...,OT,24,GRY,400.0,STOLEN,The Beaches,63,43.678585,-79.290672,1004
4,-79.380928,43.656330,1805,GO-20149004067,THEFT UNDER,2014-06-13T04:00:00.000Z,2014,6,13,18:15,...,MT,21,BLK,700.0,STOLEN,Church-Yonge Corridor,75,43.656330,-79.380928,1005


In [3]:
#Only take a look at variables related to location and status of stolen bike, drop the rest 
#Such as Division, City, Location_Type, Premise_Type, Neighbourhood, Lat, Long

df_locations = df[['Division','City','Location_Type','Premise_Type','Status','Neighbourhood','Lat','Long']].copy()
df_locations.head()

,Division,City,Location_Type,Premise_Type,Status,Neighbourhood,Lat,Long
0,51,Toronto,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,STOLEN,Moss Park,43.652412,-79.367424
1,51,Toronto,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,STOLEN,Regent Park,43.659527,-79.357460
2,51,Toronto,"Parking Lots (Apt., Commercial Or Non-Commercial)",Outside,STOLEN,Church-Yonge Corridor,43.661373,-79.383110
3,55,Toronto,"Single Home, House (Attach Garage, Cottage, Mo...",House,STOLEN,The Beaches,43.678585,-79.290672
4,51,Toronto,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,STOLEN,Church-Yonge Corridor,43.656330,-79.380928


In [4]:
#Check out the basic meta information about the data columns
df_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17892 entries, 0 to 17891
Data columns (total 8 columns):
Division         17892 non-null int64
City             17892 non-null object
Location_Type    17892 non-null object
Premise_Type     17892 non-null object
Status           17892 non-null object
Neighbourhood    17892 non-null object
Lat              17892 non-null float64
Long             17892 non-null float64
dtypes: float64(2), int64(1), object(5)
memory usage: 1.1+ MB


In [5]:
#Checking for any null values (NaN)
df_locations.isnull().values.any()

False

In [6]:
#Some simple counts based on the dataset

numBikes = len(df_locations)
numStolenBikes = len(df_locations[df_locations.Status=='STOLEN'])
numRecoveredBikes = len(df_locations[df_locations.Status=='RECOVERED'])
numUnknown = len(df_locations[df_locations.Status=='UNKNOWN'])

stolenPercentages = (numStolenBikes / numBikes) * 100
recoveredPercentages = (numRecoveredBikes / numBikes) * 100

print("Total number of bikes", numBikes)
print("Total number of stolen bikes {} with a {}%".format(numStolenBikes, stolenPercentages))
print("Total number of recovered bikes {} with a {}%".format(numRecoveredBikes, recoveredPercentages))
print("Total number of bikes with unknown status {}, % of {}".format(numUnknown, numUnknown/numBikes))

Total number of bikes 17892
Total number of stolen bikes 17347 with a 96.95394589760788%
Total number of recovered bikes 231 with a 1.2910798122065728%
Total number of bikes with unknown status 314, % of 0.017549742901855576


You can see from the above percentages they are highly skewed towards a bike being stolen and not recovered, thus it doesn't make sense to graph it stolen vs. recovered and we will continue to look at the other attributes while also dropping the status of unknown as it doesn't provide any value for the answer to our question

In [7]:
#Lets drop the status of unknown from our dataset as this provides no value in calculating if your bike has been recovered or not
df_locations = df_locations[df_locations.Status != "UNKNOWN"]

#Counts by City and Status
pd.crosstab(df_locations['City'],df_locations['Status'])

Status,RECOVERED,STOLEN
City,,
Toronto,231,17347


Above confirms the dataset only contains values for the city of Toronto and the status of unknown being dropped, no more analyis needed here

In [8]:
#Counts by Location and Status
df_location_type = pd.crosstab(df_locations['Location_Type'],df_locations['Status'], margins=True).sort_values('STOLEN',ascending=False).apply(lambda r: (r/len(df_locations['Status']))*100, axis=1)

#Lets find the top 5 location types of stolen bikes 
df_location_type.head(6)

Status,RECOVERED,STOLEN,All
Location_Type,,,
All,1.314143,98.685857,100.000000
"Streets, Roads, Highways (Bicycle Path, Private Road)",0.364091,21.316418,21.680510
"Apartment (Rooming House, Condo)",0.199113,19.313915,19.513028
"Single Home, House (Attach Garage, Cottage, Mobile)",0.227557,16.082603,16.310160
"Parking Lots (Apt., Commercial Or Non-Commercial)",0.085334,9.432245,9.517579
"Other Commercial / Corporate Places (For Profit, Warehouse, Corp. Bldg",0.113779,9.250199,9.363978


Based on the above, you can see the top 5 types of locations where bikes are stolen are 
- Streets, Roads, Highways
- Apartments
- Single Home, House
- Parking Lots
- Other commercial / corporate places

In [9]:
#Counts by Premise and Status
df_premise_type = pd.crosstab(df_locations['Premise_Type'],df_locations['Status'], margins=True).sort_values('STOLEN',ascending=False).apply(lambda r: (r/len(df_locations['Status']))*100, axis=1)

df_premise_type

Status,RECOVERED,STOLEN,All
Premise_Type,,,
All,1.314143,98.685857,100.000000
Outside,0.466492,32.449653,32.916145
Apartment,0.199113,19.313915,19.513028
Other,0.261691,18.477643,18.739333
House,0.227557,16.082603,16.310160
Commercial,0.159290,12.362043,12.521333


No surprise here if you know anything about biking in Toronto, top Premise types where bikes are stolen are 
- Outside
- Apartment
- Other 
- House 
- Commercial 

In [10]:
#Counts by Neighbourhood and Status
df_neigh = pd.crosstab(df_locations['Neighbourhood'],df_locations['Status'], margins=True).sort_values('STOLEN',ascending=False).apply(lambda r: (r/len(df_locations['Status']))*100, axis=1)

#Lets find the top 5 neighbourhoods were bikes are stolen 
df_neigh.head(6)

Status,RECOVERED,STOLEN,All
Neighbourhood,,,
All,1.314143,98.685857,100.000000
Waterfront Communities-The Island,0.113779,10.325407,10.439185
Church-Yonge Corridor,0.079645,7.475253,7.554898
Bay Street Corridor,0.062578,7.327341,7.389919
Annex,0.045511,4.232563,4.278075
University,0.034134,3.362157,3.396291


Based on the neighbourhood groupings within this dataset the top 5 are in order
- Waterfront Communities, The Island
- Church Yonge Corridor
- Bay Street Corridor
- Annex 
- University 

The Location_Type field has too many unique values, so let's see what the results are if we were to combine Neighbourhood and Premise Type

In [11]:
#df_prem_neigh = df_locations.pivot_table(index='Neighbourhood', columns='Premise_Type', values='Status', fill_value=0)
#df_prem_neigh = df_locations.groupby(['Neighbourhood', 'Premise_Type']).Status.count().unstack(fill_value=0)
df_prem_neigh = df_locations[df_locations.Status=='STOLEN']
df_prem_neigh = df_locations.pivot_table(index='Neighbourhood', columns='Premise_Type', values='Status', fill_value=0, aggfunc='count',margins=True).sort_values('All',ascending=False).apply(lambda r: (r/len(df_locations['Status']))*100, axis=1)

df_prem_neigh.head(6)

Premise_Type,Apartment,Commercial,House,Other,Outside,All
Neighbourhood,,,,,,
All,19.513028,12.521333,16.310160,18.739333,32.916145,100.000000
Waterfront Communities-The Island,3.242690,2.048015,0.073956,0.637160,4.437365,10.439185
Church-Yonge Corridor,1.217431,1.188986,0.085334,1.024007,4.039140,7.554898
Bay Street Corridor,0.785072,1.501877,0.011378,1.280009,3.811583,7.389919
Annex,0.779383,0.512004,0.699738,0.705427,1.581522,4.278075
University,0.267380,0.176357,0.426670,1.780635,0.745250,3.396291


When we combine the Premise_Type of our dataset with the combination of the nighbourhood we can break down our analysis to suggest the top 5 places you shouldn't lock up your bike if you don't want it stolen, they are 
- Outside within the Waterfront Communities / The Island
- Outside within the Church and Yonge Corridor
- Outside within the Bay Street Corridor
- Within an Apartment complex in the Waterfront Communities / The Island
- Outside within the Annex


In [16]:
#Lets graph the lat and long results to help visualize the above 
# Store our latitude and longitude
latitudes = df_locations["Lat"]
longitudes = df_locations["Long"]

gmap = gmplot.GoogleMapPlotter(43.652412, -79.367424, 13)

# Overlay our datapoints onto the map
gmap.heatmap(latitudes, longitudes)

# Generate the heatmap into an HTML file
gmap.draw("my_heatmap.html")

In [17]:
%%HTML
<iframe width="75%" height="500" src="my_heatmap.html"></iframe>

Image of the heatmap below in case the gmap function does not work within other notebooks
![title](Heatmap_Image.png)